In [ ]:
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm
import numpy as np
from   scipy.stats import norm
from scipy.stats import anderson
from math import log2
from math import sqrt
import json
import numpy as np
from scipy import stats
import math
import numpy as np
from scipy import stats

# Get annotations for each time series

In [ ]:
with open('\\TCPDBench\\analysis\\annotations\\annotations.json') as f:
    annotations = json.load(f)

# Load each time series and standarize it

In [ ]:
# DATASETS CAN BE FOUND HERE: https://github.com/alan-turing-institute/TCPDBench

DATASETS = [
#    "apple",
    "bank",
#    "bee_waggle_6",
    "bitcoin",
    "brent_spot",
    "businv",
    "centralia",
    "children_per_woman",
    "co2_canada",
    "construction",
    "debt_ireland",
    "gdp_argentina",
    "gdp_croatia",
    "gdp_iran",
    "gdp_japan",
    "global_co2",
    "homeruns",
#    "iceland_tourism",
    "jfk_passengers",
    "lga_passengers",
    "measles",
    "nile",
#    "occupancy",
    "ozone",
    "quality_control_1",
    "quality_control_2",
    "quality_control_3",
    "quality_control_4",
    "quality_control_5",
    "rail_lines",
    "ratner_stock",
    "robocalls",
#    "run_log",
    "scanline_126007",
    "scanline_42049",
    "seatbelts",
    "shanghai_license",
    "uk_coal_employ",
    "unemployment_nl",
    "usd_isk",
    "us_population",
    "well_log",
]
DATASET_NAMES = {k: k for k in DATASETS}

In [ ]:
timeSeriesData = []

for dataset in DATASETS:
    filename = '..\\TCPD\\datasets\\{}\\'.format(dataset)
    with open(filename+'{}.json'.format(dataset)) as f:
        timeSeriesData.append(json.load(f))

In [ ]:
stdTestingData = []

for i in range(len(timeSeriesData)):
    rawData = timeSeriesData[i]['series'][0]['raw']
    if None in rawData:
        indexOfNone = [u for u,v in enumerate(rawData) if v == None]
        for iN in indexOfNone:
            rawData[iN] = (rawData[iN-1] + rawData[iN+1]) / 2

    rawData = np.asarray(rawData)
    rawData = (rawData - np.mean(rawData)) / np.std(rawData)
    stdTestingData.append(rawData)
    print(len(stdTestingData[i]))
    if(len(stdTestingData[i])< 40):
        print(DATASETS[i])

stdTestingData = np.asarray(stdTestingData)

# Define Covering and F1-Score 

In [ ]:
def true_positives(T, X, margin=5):
    """Compute true positives without double counting

    >>> true_positives({1, 10, 20, 23}, {3, 8, 20})
    {1, 10, 20}
    >>> true_positives({1, 10, 20, 23}, {1, 3, 8, 20})
    {1, 10, 20}
    >>> true_positives({1, 10, 20, 23}, {1, 3, 5, 8, 20})
    {1, 10, 20}
    >>> true_positives(set(), {1, 2, 3})
    set()
    >>> true_positives({1, 2, 3}, set())
    set()
    """
    # make a copy so we don't affect the caller
    X = set(list(X))
    TP = set()
    for tau in T:
        close = [(abs(tau - x), x) for x in X if abs(tau - x) <= margin]
        close.sort()
        if not close:
            continue
        dist, xstar = close[0]
        TP.add(tau)
        X.remove(xstar)
    return TP


def f_measure(annotations, predictions, margin=5, alpha=0.5, return_PR=False):
    """Compute the F-measure based on human annotations.

    annotations : dict from user_id to iterable of CP locations
    predictions : iterable of predicted CP locations
    alpha : value for the F-measure, alpha=0.5 gives the F1-measure
    return_PR : whether to return precision and recall too

    Remember that all CP locations are 0-based!

    >>> f_measure({1: [10, 20], 2: [11, 20], 3: [10], 4: [0, 5]}, [10, 20])
    1.0
    >>> f_measure({1: [], 2: [10], 3: [50]}, [10])
    0.9090909090909091
    >>> f_measure({1: [], 2: [10], 3: [50]}, [])
    0.8
    """
    # ensure 0 is in all the sets
    Tks = {k + 1: set(annotations[uid]) for k, uid in enumerate(annotations)}
    for Tk in Tks.values():
        Tk.add(0)

    X = set(predictions)
    X.add(0)

    Tstar = set()
    for Tk in Tks.values():
        for tau in Tk:
            Tstar.add(tau)

    K = len(Tks)

    P = len(true_positives(Tstar, X, margin=margin)) / len(X)

    TPk = {k: true_positives(Tks[k], X, margin=margin) for k in Tks}
    R = 1 / K * sum(len(TPk[k]) / len(Tks[k]) for k in Tks)

    F = P * R / (alpha * R + (1 - alpha) * P)
    if return_PR:
        return F, P, R
    return F


def overlap(A, B):
    """ Return the overlap (i.e. Jaccard index) of two sets

    >>> overlap({1, 2, 3}, set())
    0.0
    >>> overlap({1, 2, 3}, {2, 5})
    0.25
    >>> overlap(set(), {1, 2, 3})
    0.0
    >>> overlap({1, 2, 3}, {1, 2, 3})
    1.0
    """
    return len(A.intersection(B)) / len(A.union(B))


def partition_from_cps(locations, n_obs):
    """ Return a list of sets that give a partition of the set [0, T-1], as 
    defined by the change point locations.

    >>> partition_from_cps([], 5)
    [{0, 1, 2, 3, 4}]
    >>> partition_from_cps([3, 5], 8)
    [{0, 1, 2}, {3, 4}, {5, 6, 7}]
    >>> partition_from_cps([1,2,7], 8)
    [{0}, {1}, {2, 3, 4, 5, 6}, {7}]
    >>> partition_from_cps([0, 4], 6)
    [{0, 1, 2, 3}, {4, 5}]
    """
    T = n_obs
    partition = []
    current = set()

    all_cps = iter(sorted(set(locations)))
    cp = next(all_cps, None)
    for i in range(T):
        if i == cp:
            if current:
                partition.append(current)
            current = set()
            cp = next(all_cps, None)
        current.add(i)
    partition.append(current)
    return partition


def cover_single(Sprime, S):
    """Compute the covering of a segmentation S by a segmentation Sprime.

    This follows equation (8) in Arbaleaz, 2010.

    >>> cover_single([{1, 2, 3}, {4, 5}, {6}], [{1, 2, 3}, {4, 5, 6}])
    0.8333333333333334
    >>> cover_single([{1, 2, 3, 4}, {5, 6}], [{1, 2, 3, 4, 5, 6}])
    0.6666666666666666
    >>> cover_single([{1, 2}, {3, 4}, {5, 6}], [{1, 2, 3}, {4, 5, 6}])
    0.6666666666666666
    >>> cover_single([{1, 2, 3, 4, 5, 6}], [{1}, {2}, {3}, {4, 5, 6}])
    0.3333333333333333
    """
    T = sum(map(len, Sprime))
    assert T == sum(map(len, S))
    C = 0
    for R in S:
        C += len(R) * max(overlap(R, Rprime) for Rprime in Sprime)
    C /= T
    return C


def Covering(annotations, predictions, n_obs):
    """Compute the average segmentation covering against the human annotations.

    annotations : dict from user_id to iterable of CP locations
    predictions : iterable of predicted Cp locations
    n_obs : number of observations in the series

    >>> covering({1: [10, 20], 2: [10], 3: [0, 5]}, [10, 20], 45)
    0.7962962962962963
    >>> covering({1: [], 2: [10], 3: [40]}, [10], 45)
    0.7954144620811286
    >>> covering({1: [], 2: [10], 3: [40]}, [], 45)
    0.8189300411522634

    """
    Ak = {
        k + 1: partition_from_cps(annotations[uid], n_obs)
        for k, uid in enumerate(annotations)
    }
    pX = partition_from_cps(predictions, n_obs)

    Cs = [cover_single(pX, Ak[k]) for k in Ak]
    return sum(Cs) / len(Cs)

# SB-BOCPD

In [ ]:
class BayesianOnlineChangePointDetection_Segment:
    def __init__(self, hazard, distribution):
        self.hazard = hazard
        self.distribution = distribution
        self.T = 0
        self.beliefs = np.zeros((1, 2),dtype=np.float64)
        self.beliefs[0, 0] = 1.0

    def reset_params(self):
        self.T = 0
        self.beliefs = np.zeros((1, 2))
        self.beliefs[0, 0] = 1.0

    def _expand_belief_matrix(self):
        rows = np.zeros((1, 2),dtype=np.float64)
        self.beliefs = np.concatenate((self.beliefs, rows), axis=0)

    def _shift_belief_matrix(self):
        self.beliefs[:, 0] = self.beliefs[:, 1]
        self.beliefs[:, 1] = 0.0

    def update(self, x, studentTQueue, historicalData):
        
        
        # Evaluate Predictive Probability (3 in Algortihm 1)
        if type(x) is not np.ndarray:
            self._expand_belief_matrix()

            self.distribution.update_params(x)
            # Update internal state
            #print(len(self.beliefs))

            self.T += 1
        else:
            self._expand_belief_matrix()
            


            pi_t = self.distribution.pdf(x, studentTQueue, historicalData)
            # Calculate H(r_{t-1})
            h = self.hazard(self.rt)

            self.beliefs[1 : self.T + 2, 1] = self.beliefs[: self.T + 1, 0] * pi_t * (1 - h)

            # Calculate Changepoint Probabilities (5 in Algorithm 1)
            self.beliefs[0, 1] = (self.beliefs[: self.T + 1, 0] * pi_t * h).sum()

            # Determine Run length Distribution (7 in Algorithm 1)
            self.beliefs[:, 1] = self.beliefs[:, 1] / self.beliefs[:, 1].sum()

            # Update sufficient statistics (8 in Algorithm 8)
            self.distribution.update_params(x[0])

            # Update internal state
            self._shift_belief_matrix()
            self.T += 1
            
            
        
        
    @property
    def rt(self):
       # return np.where(self.beliefs[:, 0] == self.beliefs[:, 0].max())[0]
    
        rtPick = np.where(self.beliefs[:, 0] > 0.5)[0]
        #print(rtPick)
        if(len(rtPick) == 0):
            return 0
        else:
            return rtPick[-1]
    @property
    def belief(self):
        return self.beliefs[:, 0]

class Hazard:
    def __call__(self, *args, **kwargs):
        raise NotImplementedError()


class ConstantHazard(Hazard):
    def __init__(self, _lambda):
        self._lambda = _lambda

    def __call__(self, r):
        """
        Args:
          r: The length of the current run (np.ndarray or scalar)

        Returns:
          p: Changepoint Probabilities(np.ndarray with shape = r.shape)
        """
        if isinstance(r, np.ndarray):
            shape = r.shape
        else:
            shape = 1

        return np.ones(shape) / self._lambda

class Distribution:
    def reset_params(self):
        raise NotImplementedError()

    def pdf(self, x):
        raise NotImplementedError()

    def update_params(self, x):
        raise NotImplementedError()


class StudentT(Distribution):
    """ Generalized Student t distribution 
    https://en.wikipedia.org/wiki/Student%27s_t-distribution#Generalized_Student's_t-distribution

    This setting corresponds to select
      1: Gaussian distribution as a likelihood
      2: normal-Gamma distribution as a prior for Gaussian
    """

    def __init__(self, mu=0, kappa=1, alpha=1, beta=1):
        self.mu0 = np.array([mu],dtype=np.float64)
        self.kappa0 = np.array([kappa],dtype=np.float64)
        self.alpha0 = np.array([alpha],dtype=np.float64)
        self.beta0 = np.array([beta],dtype=np.float64)
        # We need the following lines to prevent "outside defined warning"
        self.muT = self.mu0.copy()
        self.kappaT = self.kappa0.copy()
        self.alphaT = self.alpha0.copy()
        self.betaT = self.beta0.copy()

    def reset_params(self):
        self.muT = self.mu0.copy()
        self.kappaT = self.kappa0.copy()
        self.alphaT = self.alpha0.copy()
        self.betaT = self.beta0.copy()

    def pdf(self, x, studentTQueue, historicalData):
        """ Probability Density Function
        """

        
        return stats.t.pdf(
            np.average(x),
            loc=self.muT,
            df=2 * self.alphaT,
            scale=np.sqrt(self.betaT * (self.kappaT + 1) / (self.alphaT * self.kappaT)),
        )

    def update_params(self, x):
        """Update Sufficient Statistcs (Parameters)

        To understand why we use this, see e.g.
        Conjugate Bayesian analysis of the Gaussian distribution, Kevin P. Murphy∗
        https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf
        3.5 Posterior predictive
        """
        self.betaT = np.concatenate(
            [
                self.beta0,
                (self.kappaT + (self.kappaT * (x - self.muT) ** 2) / (2 * (self.kappaT + 1))),
            ]
        )
        self.muT = np.concatenate([self.mu0, (self.kappaT * self.muT + x) / (self.kappaT + 1)])
        self.kappaT = np.concatenate([self.kappa0, self.kappaT + 1])
        self.alphaT = np.concatenate([self.alpha0, self.alphaT + 0.5])



# Test SB-BOCPD

In [ ]:
counter = 0
fMeasures = np.zeros(len(DATASETS))
coverings = np.zeros(len(DATASETS))


alpha = [0.01,1,100]
beta = [0.01,1,100]
kappa = [0.01,1,100]
hazard = [50,100,200]
segmentLength = [3,5,8,13,21,34]

for a in alpha:
    for b in beta:
        for k in kappa:
            for l in segmentLength:
                for n,datasetName in enumerate(DATASETS):
                    studentTQueue = []

                    bc = BayesianOnlineChangePointDetection_Segment(ConstantHazard(300), StudentT(mu=0, kappa=k, alpha=a, beta=b))
                    
                    
                    testSignal=stdTestingData[11]
                    historicalData = []
                    rt_mle = np.empty(testSignal.shape)
                    L = l

                    cuttoff = math.floor(len(testSignal)/50)
                    allData = []

                    # Online estimation and get the maximum likelihood r_t at each time point
                    for i, d in enumerate(testSignal):
                        #print(d)
                        #print(testSignal[0:20])
                        allData.append(d)
                        if i >= L:
                            for j in range(len(historicalData)):
                                historicalData[j].append(testSignal[i-L])
                            historicalData.insert(0,[testSignal[i-L]])
                            localData = np.asarray(allData[-L:])
                            
                            bc.update(localData,studentTQueue,historicalData)
                            rt_mle[i] = bc.rt
                        else: 
                            rt_mle[i] = 0

                    plt.plot(rt_mle)
                    plt.show()
                    # Plot data with estimated change points
                    plt.plot(testSignal, alpha=0.5, label="observation")
                    
                    index_changes = np.where(np.diff(rt_mle)<-cuttoff)[0]-int(L/2)
                    if(f_measure(annotations[datasetName], index_changes) == 1.0):
                        print(l)
                        print('********************************')
                        continue
                    print(index_changes)
                    plt.scatter(index_changes, testSignal[index_changes], c='green', label="change point")
                    plt.show()
                    
                    print('alpha' + str(a))
                    print('beta' + str(b))
                    print('kappa' + str(k))
                    
                    print(datasetName)
                    
                    print("F-Measure")
                    print(f_measure(annotations[datasetName], index_changes))
                    
                    fMeasure = f_measure(annotations[datasetName], index_changes)
                    if fMeasure > fMeasures[n]:
                        fMeasures[n] = fMeasure
                   
                    print("Covering")
                    covering = Covering(annotations[datasetName], index_changes,len(testSignal))
                    print(covering)
                    if covering > coverings[n]:
                        coverings[n] = covering
                        
for c in coverings:
    print(c)
print()
for c in fMeasures:
    print(c)                      

